In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import sys
sys.path.append("../..")

In [3]:
import pandas as pd
import numpy as np
import pipit as pp

In [4]:
trace = pp.Trace.from_otf2("../../pipit/tests/data/ping-pong-otf2/")
pandas_df = trace.events[["Timestamp (ns)", "Name", "Event Type", "Process", "Thread"]].copy()
pandas_df["Process"] = pandas_df["Process"].astype(int)
pandas_df["Thread"] = pandas_df["Thread"].astype(int)

In [55]:
from pipit.util.config import set_option, get_option

set_option("backend", "pandas")
get_option("backend")

'pandas'

In [61]:
from pipit.dsl.dataset import create_dataset

ds = create_dataset()

In [63]:
ds.show()

(Empty dataset)



In [64]:
# simulating a reader
row = pandas_df.iloc[0]
evt = Event(timestamp=row["Timestamp (ns)"], process=row["Process"], thread=row["Thread"], name=row["Name"], event_type=row["Event Type"])
ds.push_event(evt)

In [65]:
ds.show()

(Empty dataset)



In [66]:
ds.flush()
ds.show()

+----+-------------+--------------+--------------+-----------+----------+
|    |   timestamp | name         | event_type   |   process |   thread |
|----+-------------+--------------+--------------+-----------+----------|
|  0 |           0 | ProgramBegin | Instant      |         1 |        0 |
+----+-------------+--------------+--------------+-----------+----------+


In [68]:
for i in range(1, len(pandas_df)):
    row = pandas_df.iloc[i]
    evt = Event(timestamp=row["Timestamp (ns)"], process=row["Process"], thread=row["Thread"], name=row["Name"], event_type=row["Event Type"])
    ds.push_event(evt)
ds.flush()

In [69]:
ds.show()

+-----+------------------+-----------------------+--------------+-----------+----------+
|     |        timestamp | name                  | event_type   |   process |   thread |
|-----+------------------+-----------------------+--------------+-----------+----------|
|   0 |      0           | ProgramBegin          | Instant      |         1 |        0 |
|   0 |  30083.1         | int main(int, char**) | Enter        |         1 |        0 |
|   1 |  40288.3         | MPI_Init              | Enter        |         1 |        0 |
|   2 | 307731           | ProgramBegin          | Instant      |         0 |        0 |
|   3 | 336980           | int main(int, char**) | Enter        |         0 |        0 |
|   4 | 346055           | MPI_Init              | Enter        |         0 |        0 |
|   5 |      1.93643e+08 | MPI_Init              | Leave        |         0 |        0 |
|   6 |      1.93644e+08 | MPI_Init              | Leave        |         1 |        0 |
|   7 |      1.93652e

In [72]:
ds.filter("process == 0").show()

+-----+------------------+-----------------------+--------------+-----------+----------+
|     |        timestamp | name                  | event_type   |   process |   thread |
|-----+------------------+-----------------------+--------------+-----------+----------|
|   2 | 307731           | ProgramBegin          | Instant      |         0 |        0 |
|   3 | 336980           | int main(int, char**) | Enter        |         0 |        0 |
|   4 | 346055           | MPI_Init              | Enter        |         0 |        0 |
|   5 |      1.93643e+08 | MPI_Init              | Leave        |         0 |        0 |
|   7 |      1.93652e+08 | MPI_Comm_size         | Enter        |         0 |        0 |
|   9 |      1.93653e+08 | MPI_Comm_size         | Leave        |         0 |        0 |
|  11 |      1.93656e+08 | MPI_Comm_rank         | Enter        |         0 |        0 |
|  13 |      1.93657e+08 | MPI_Comm_rank         | Leave        |         0 |        0 |
|  15 |      1.93668e